In [1]:
# Installing required package
!pip install yfinance --quiet

# Importing essential libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Displaying all columns while printing DataFrames
pd.set_option('display.max_columns', None)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Selecting 10 liquid stocks from different sectors (NSE)
tickers = [
    'INFY.NS', 'TCS.NS', 'RELIANCE.NS', 'HDFCBANK.NS',
    'ITC.NS', 'LT.NS', 'HINDUNILVR.NS', 'SBIN.NS',
    'BHARTIARTL.NS', 'MARUTI.NS'
]

# Print tickers for confirmation
print("Selected tickers:", tickers)

In [ ]:
# Downloading 10 years of historical daily data correctly for adjusted close prices
data = yf.download(tickers, start='2015-01-01', end='2025-01-01', auto_adjust=True)

# 'auto_adjust=True' means downloaded data is already adjusted for splits/dividends, so we can directly use the 'Close' column and the result will already be a DataFrame of Close prices

# Displaying the first few rows
data.head()


In [ ]:
# Checking missing values
print("Missing values in each stock:")
print(data.isna().sum())

# Dropping rows with missing values
data = data.dropna()

# Confirming dataset shape after cleaning
print("\nCleaned data shape:", data.shape)

In [ ]:
# Computing log returns
log_returns = np.log(data["Close"] / data.shift(1)["Close"]).dropna()
# Displaying first few rows of log returns
log_returns.head()

In [ ]:
# Defining periods
in_sample = log_returns.loc['2015-01-01':'2020-12-31']
out_sample = log_returns.loc['2021-01-01':'2022-12-31']

# Printing shapes to confirm
print("In-sample data shape:", in_sample.shape)
print("Out-of-sample data shape:", out_sample.shape)

# Checking the time ranges
print("\nIn-sample range:", in_sample.index.min(), "to", in_sample.index.max())
print("Out-of-sample range:", out_sample.index.min(), "to", out_sample.index.max())

In [ ]:
# Plotting adjusted close prices
data["Close"].plot(figsize=(12, 6), title='Adjusted Close Prices (2015-2025)')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)
plt.show()

# Plotting log returns to inspect volatility
log_returns.plot(figsize=(12, 6), title='Daily Log Returns')
plt.xlabel('Date')
plt.ylabel('Log Return')
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns

In [ ]:

# Part A.1 - Computing sample statistics for in‑sample data

# Mean vector (daily)
mean_daily = in_sample.mean()

# Covariance matrix (daily)
cov_daily = in_sample.cov()
print("cov_daily", cov_daily.shape)
print(cov_daily)

# Correlation matrix
corr_matrix = in_sample.corr()

#  Annualizing (because these are daily log‑returns)
#  Convention: 252 trading days per year
mean_annual = mean_daily * 252
cov_annual = cov_daily * 252
std_annual = np.sqrt(np.diag(cov_annual))

print(" Mean return vector (annualized):")
display(mean_annual.to_frame("Expected Annual Return"))

print("\n Covariance matrix (annualized):")
display(cov_annual)

print("\n Correlation matrix:")
display(corr_matrix)

In [ ]:
# Computing descriptive statistics for each stock
summary_table = pd.DataFrame({
    "Average Annual Return": mean_annual,
    "Annual Std Dev": std_annual,
    "Skewness": in_sample.skew(),
    "Kurtosis": in_sample.kurt()
})

# Rounding for better readability
summary_table = summary_table.round(4)

print("Descriptive Statistics — In‑Sample Period (2015–2020):")
display(summary_table)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Pairwise Correlation Matrix — In‑Sample Period (2015–2020)")
plt.show()

In [ ]:
# Ensuring clean numeric data before fitting the covariance estimator

# Replacing infinite values with NaN
in_sample_clean = in_sample.replace([np.inf, -np.inf], np.nan)

# Dropping NaN rows (or you could fill them if very small number)
in_sample_clean = in_sample_clean.dropna()

# Checking that there are no issues left
print("Any NaN left?", in_sample_clean.isna().sum().sum())
print("Shape after cleaning:", in_sample_clean.shape)

In [ ]:
# ============================================================
# Part B: Alternative Estimator Implementation — Ledoit–Wolf Shrinkage
# ============================================================

# Objective:
# We are implementing the Ledoit–Wolf shrinkage method to estimate
# a more stable and robust covariance matrix for asset returns.
#
# Formula conceptually:
# Σ_shrink = (1 - λ) * Σ_sample + λ * F
# where:
#   - Σ_sample: sample covariance matrix
#   - F: shrinkage target (identity matrix scaled by average variance)
#   - λ: shrinkage intensity (automatically determined)
#
# Ledoit–Wolf built-in algorithm automatically computes F and λ.


# Step 1: Importing the estimator

from sklearn.covariance import LedoitWolf


# Step 2: Initializing the Ledoit–Wolf shrinkage object

lw = LedoitWolf()


# Step 3: Fitting the estimator on our cleaned in-sample log returns

# Using 'in_sample_clean' ensures that we have no NaN or infinite values.
lw.fit(in_sample_clean)


# Step 4: Extracting the estimated covariance matrix (daily)

cov_lw_daily = lw.covariance_


# Step 5: Annualizing the covariance matrix

# Convention used: 252 trading days per year. So, annual covariance = daily covariance * 252
cov_lw_annual = cov_lw_daily * 252


# Step 6: Converting the numpy array to a well-labeled DataFrame

# This helps with readability, visualization, and comparison
cov_lw_df = pd.DataFrame(
    cov_lw_annual,
    index=in_sample_clean.columns,
    columns=in_sample_clean.columns
)

# Step 7: Displaying the shrinkage covariance matrix

print("Ledoit–Wolf Shrinkage Covariance Matrix (Annualized):")
display(cov_lw_df)


# Step 8:Validation and Quick Summary and also checking for any anomalies or unreasonable covariance values
print("Matrix shape:", cov_lw_df.shape)
print("Minimum covariance value:", np.min(cov_lw_df.values))
print("Maximum covariance value:", np.max(cov_lw_df.values))



In [ ]:
# Computing difference between covariance matrices
diff_cov = cov_annual - cov_lw_df

print(" Difference between Sample and Ledoit‑Wolf Covariance (annualized):")
display(diff_cov.round(6))

# Comparing average variance and correlation magnitude
avg_var_sample = np.mean(np.diag(cov_annual))
avg_var_lw = np.mean(np.diag(cov_lw_df))

print(f"\nAverage variance (Sample): {avg_var_sample:.6f}")
print(f"Average variance (Ledoit-Wolf): {avg_var_lw:.6f}")

In [ ]:

# Ensurimg both sample and Ledoit–Wolf covariance are clean


# Reconfirming that 'in_sample_clean' has no missing or infinite values
print("Checking data quality before covariance computations:")
print("Any NaN or Inf left in data? ->", 
      in_sample_clean.replace([np.inf, -np.inf], np.nan).isna().sum().sum())

# Computing clean sample-based covariance (daily)
cov_daily_clean = in_sample_clean.cov()

# Annualizing sample covariance (same convention)
# Convention reminder:
#   - Mean annual = daily mean × 252
#   - Covariance annual = daily covariance × 252
cov_annual_clean = cov_daily_clean * 252

# Computing average variance (diagonals)
avg_var_sample = np.mean(np.diag(cov_annual_clean))
avg_var_lw = np.mean(np.diag(cov_lw_df))

# Printing to compare
print("\n Comparison of Average Variance (Annualized):")
print(f"Average variance (Sample, clean): {avg_var_sample:.6f}")
print(f"Average variance (Ledoit-Wolf):  {avg_var_lw:.6f}")

# Sanity check difference
if np.isnan(avg_var_sample):
    print(" Warning: Sample covariance still contains NaNs — check cleaning steps.")
else:
    print(" Both covariance matrices are now clean and comparable.")

In [ ]:
# FIXING MultiIndex problem 

if isinstance(cov_annual_clean.columns, pd.MultiIndex):
    cov_annual_clean.columns = cov_annual_clean.columns.get_level_values(-1)
    cov_annual_clean.index = cov_annual_clean.index.get_level_values(-1)
    print("Flattened MultiIndex in covariance matrix.")
else:
    print("Covariance matrix already has flat column/index names.")



In [ ]:

# Part C — Diversification and Portfolio Variance

# Goal:
#   - For k = 1 ... 10 stocks, compute portfolio variance of
#     all equal‑weighted combinations.
#   - Recording mean, 10th, 50th, 90th percentiles for each k.
#   - Visualizing diversification effect.

import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Confirming we have the cleaned covariance matrix from Part B
print(" Checking covariance matrix you will use:")
print(f"Matrix dimensions: {cov_annual_clean.shape}")
print(f"Ticker count: {len(cov_annual_clean.columns)}")
print("Ticker list:", list(cov_annual_clean.columns))
print("----------------------------------------------------\n")

# Confirming annualization convention
print(" Annualization convention: 252 trading days/year")
print("All portfolio variances computed in ANNUALIZED terms.\n")

In [ ]:

# proper shape handling


def portfolio_variance(cov_matrix, subset_tickers):
    """
    Computes variance of an equally weighted portfolio for a given subset of tickers.
    Handles both single- and multi-asset subsets safely.
    -------------------------------------------------------------
    Parameters:
        cov_matrix: Pandas DataFrame (annualized covariance matrix)
        subset_tickers: list/tuple of selected stock tickers
    Returns:
        variance (float)
    -------------------------------------------------------------
    """
    # Ensuring subset_tickers is always a list even when a single asset is passed
    if isinstance(subset_tickers, str):
        subset_tickers = [subset_tickers]
    if isinstance(subset_tickers, tuple):
        subset_tickers = list(subset_tickers)

    k = len(subset_tickers)

    # Extracting the covariance submatrix as a numpy array
    sub_cov = cov_matrix.loc[subset_tickers, subset_tickers].to_numpy()

    # Constructing equal weights: 1/k each
    w = np.repeat(1 / k, k).reshape(k, 1)

    # Computing portfolio variance = w'Σw
    var_p = float(np.dot(w.T, np.dot(sub_cov, w)))

    # Debugging prints for small subsets (so we can see progress)
    if k <= 2:
        print(f"Subset: {subset_tickers}, Variance: {var_p:.6f}")
    
    return var_p

In [ ]:

# Collapsing duplicate ticker labels in covariance matrix

print("Original shape:", cov_annual_clean.shape)

# 1️Grouping by ticker name on both rows and columns and take the mean
cov_fixed = (
    cov_annual_clean.groupby(level=0, axis=0).mean()
                    .groupby(level=0, axis=1).mean()
)

# Replacing the old matrix with the new clean one
cov_annual_clean = cov_fixed.copy()

# Verifying new shape and contents
print(" Cleaned covariance matrix to unique tickers only.")
print("New shape:", cov_annual_clean.shape)
print("Ticker list:", list(cov_annual_clean.columns))

In [ ]:
tickers_list = list(cov_annual_clean.columns)
results = []

for k in range(1, len(tickers_list) + 1):
    subset_vars = []
    combs = list(itertools.combinations(tickers_list, k))
    print(f"Computing {len(combs)} combinations for k = {k} ...")
    
    for subset in combs:
        var_p = portfolio_variance(cov_annual_clean, subset)
        subset_vars.append(var_p)
    
    mean_var = np.mean(subset_vars)
    p10 = np.percentile(subset_vars, 10)
    p50 = np.percentile(subset_vars, 50)
    p90 = np.percentile(subset_vars, 90)
    
    print(f"Summary for k={k}:")
    print(f"  Mean : {mean_var:.6f}")
    print(f"  10th : {p10:.6f}")
    print(f"  50th : {p50:.6f}")
    print(f"  90th : {p90:.6f}\n")
    
    results.append({"k": k, "mean_variance": mean_var, "p10": p10, "p50": p50, "p90": p90})

diversification_df = pd.DataFrame(results)
print("Finished computing all portfolio variances.\n")
display(diversification_df)

In [ ]:

# Visualization: Diversification Effect — Portfolio Variance vs. Number of Stocks
plt.figure(figsize=(12, 7))

# Plotting mean portfolio variance (solid line with markers)
plt.plot(
    diversification_df["k"],
    diversification_df["mean_variance"],
    color="navy",
    marker="o",
    linewidth=2.0,
    markersize=8,
    label="Mean Portfolio Variance"
)


#  Shaded area for dispersion (10th–90th percentile)

plt.fill_between(
    diversification_df["k"],
    diversification_df["p10"],
    diversification_df["p90"],
    color="skyblue",
    alpha=0.3,
    label="10th–90th Percentile Range"
)


# Median variance line for additional clarity!

plt.plot(
    diversification_df["k"],
    diversification_df["p50"],
    linestyle="--",
    color="gray",
    linewidth=2,
    label="Median Portfolio Variance (50th %)"
)


plt.text(
    2, diversification_df["mean_variance"].max() * 0.85,
    "Variance falls rapidly as assets are added\n"
    "→ Diversification reduces risk",
    fontsize=12,
    color="black",
    bbox=dict(boxstyle="round,pad=0.4", facecolor="white", edgecolor="gray", alpha=0.7)
)


# Labels, title, and formatting

plt.title(
    "Diversification Effect: Portfolio Variance vs. Number of Stocks (Equally Weighted)",
    fontsize=14,
    fontweight="bold"
)
plt.xlabel("Number of Stocks in Portfolio (k)", fontsize=12)
plt.ylabel("Mean Annualized Portfolio Variance", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(fontsize=11, loc="upper right")
plt.tight_layout()


# Showing the chart

plt.show()


#  Printing interpretation 

print("\n Interpretation:")
print("• The blue line shows the mean (average) portfolio variance for equally weighted portfolios of size k.")
print("• The shaded blue band shows the spread between the 10th and 90th percentiles across all possible combinations.")
print("• As k increases, the mean variance declines sharply at first and then flattens.")
print("• The narrowing shaded band confirms that larger portfolios have more stable risk.")
print("\n This plot visually confirms the diversification effect —")
print("   adding more assets lowers the portfolio’s total risk but with diminishing marginal benefit.")